# Classification with Decision Trees

In this notebook, I will implement a meachine learning algorithm called "decision tree", and use it to build a model from the historical data of medical patients and their response to different medications. The model will be used to predict the class of an unkown patient, or to find the proper medication for a new one.

## 1. Preliminaries: basic modules and dataset

We begin by loading the most basic packages: `pandas` to work with rich dataframe objects and `numpy` to work with arrays.

In [2]:
import numpy as np
import pandas as pd

The modelling problem we face in this notebook concerns data about medical patients with the same illness. Each patient responded will to one out of five different medicies, called drugs A, B, C, X and Y.

The goal is to build a model that effectively predicts, based on health data, which drug will be effective on a patient with the same illness. The variables collected in the dataset are `Age`, `Sex`, `Blood Pressure`, and `Cholesterol`of the patients. The variable to be predicted is the drug that works well on the patient.
This is an example about classification with several classes. We will build a "decision tree classifier".

## 2. Loading the data

In [4]:
df = pd.read_csv("drug-dataset.csv", delimiter = ",")
df.head()

Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  drugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  drugY

We will need to preprocess this dataframe object. For example, the `Sex`, `BP`, and `Cholesterol` are categorical, and their values are all strings. We should encode these as numerical variables. We begin by inspecting how much data we have. The number of rows is the number of patients or observations:

In [5]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of attributes: {df.shape[1]}")

Number of rows: 200
Number of attributes: 6


## 3. Preprocessing: encoding and splitting

Instead of a `pandas` DataFrame object we want a `numpy` array, because the `sklearn` implementation of the decision tree classifier is trained upon arrays. `numpy` is not bothered by having strings in array entries. We first construct the array of observation features.

In [12]:
features = ["Age", "Sex", "BP", "Cholesterol", "Na_to_K"]
X = df[features].values
X[:2,:]  #we print the first two observations

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093]], dtype=object)

The target variable vector is also translated into a `numpy` array.

In [13]:
y = df[['Drug']].values
y[:2]

array([['drugY'],
       ['drugC']], dtype=object)

We now want to transform categorical string variables into numerical ones. This is done with an object called `LabelEncoder` from the `sklearn.preprocessing` module. It transforms the finite set of values of the `Sex` feature into integer numbers.

In [14]:
from sklearn import preprocessing

le_sex = preprocessing.LabelEncoder()
le_sex.fit(["F","M"]) # This receives the strings that it should associate to categorical data to "train" the encoder


X[:,1] = le_sex.transform(X[:,1])
X[:5]

array([[23, 0, 'HIGH', 'HIGH', 25.355],
       [47, 1, 'LOW', 'HIGH', 13.093],
       [47, 1, 'LOW', 'HIGH', 10.114],
       [28, 0, 'NORMAL', 'HIGH', 7.798],
       [61, 0, 'LOW', 'HIGH', 18.043]], dtype=object)

Now `0` in the second column corresponds to `F` (female) and `1` corresponds to `M` (male).
We do the same with the `BP`, the blood pressure variable, and with the `Cholesterol` variable. The same encoder works, actually, as both features take exactly the same values.

In [15]:
le_bp = preprocessing.LabelEncoder()
le_bp.fit(['LOW','NORMAL','HIGH'])
X[:,2] = le_bp.transform(X[:,2])
X[:,3] = le_bp.transform(X[:,3])

We now have to split the dataset into training and testing. This is easily done with the `train_test_split()` function from the `sklearn.model_selection` module.

In [18]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size = 0.2)

print(f"Size of train sample: {X_train.shape[0]}")
print(f"Size of test sample: {X_test.shape[0]}")

Size of train sample: 160
Size of test sample: 40


## 4. Setting up and training the model

We now create an instance of the DecisionTreeClassifier class. We will specify the criterion for training as "entropy" instead of the usual Gini coefficient. Also you need to tell the decision tree how many layers of maximum depth it can have (otherwise you will be able to classify anything you want with a huge overfit...).

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion = "entropy", max_depth = 4)

Now we simply train the model on the training dataset.

In [21]:
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

## 5. Prediction

We now want to be able to predict which drug will be useful for the patients/observations in the testing dataset. We store these predictions in the `y_hat` variable

In [22]:
y_hat = model.predict(X_test)
y_hat.shape

(40,)

Let's print some of the predictions against the actual results:

In [24]:
print('Actual medicine that worked:')
print(np.array(y_test).reshape(40,)[0:10])
print('Predicted medicine from the decision tree classifier:')
print(y_hat[0:10])

Actual medicine that worked:
['drugB' 'drugB' 'drugA' 'drugX' 'drugY' 'drugX' 'drugY' 'drugX' 'drugY'
 'drugY']
Predicted medicine from the decision tree classifier:
['drugB' 'drugB' 'drugA' 'drugX' 'drugY' 'drugX' 'drugY' 'drugX' 'drugY'
 'drugY']


## 6. Testing the model

With the `metrics` submodule we can use the `accuracy_score()` function to check how well the model generalizes to the rest of the test dataset.

In [25]:
from sklearn import metrics

print(f"Model accuracy: {metrics.accuracy_score(y_test, y_hat)}")

Model accuracy: 0.975


## 7. Visualization

Decision trees are very interpretable models, as far from a black box as one can get in machine learning, and there are some nice visualization tools to check which tests the decision tree classifier model implements.

In [28]:
from sklearn import tree

tree.plot_tree(model,feature_names = features, class_names = model.classes_ )

[Text(0.625, 0.9, 'Na_to_K <= 14.615\nentropy = 1.954\nsamples = 160\nvalue = [17, 12, 15, 40, 76]\nclass = drugY'),
 Text(0.5, 0.7, 'BP <= 0.5\nentropy = 1.821\nsamples = 84\nvalue = [17, 12, 15, 40, 0]\nclass = drugX'),
 Text(0.25, 0.5, 'Age <= 50.5\nentropy = 0.978\nsamples = 29\nvalue = [17, 12, 0, 0, 0]\nclass = drugA'),
 Text(0.125, 0.3, 'entropy = 0.0\nsamples = 17\nvalue = [17, 0, 0, 0, 0]\nclass = drugA'),
 Text(0.375, 0.3, 'entropy = 0.0\nsamples = 12\nvalue = [0, 12, 0, 0, 0]\nclass = drugB'),
 Text(0.75, 0.5, 'BP <= 1.5\nentropy = 0.845\nsamples = 55\nvalue = [0, 0, 15, 40, 0]\nclass = drugX'),
 Text(0.625, 0.3, 'Cholesterol <= 1.0\nentropy = 0.991\nsamples = 27\nvalue = [0, 0, 15, 12, 0]\nclass = drugC'),
 Text(0.5, 0.1, 'entropy = 0.0\nsamples = 15\nvalue = [0, 0, 15, 0, 0]\nclass = drugC'),
 Text(0.75, 0.1, 'entropy = 0.0\nsamples = 12\nvalue = [0, 0, 0, 12, 0]\nclass = drugX'),
 Text(0.875, 0.3, 'entropy = 0.0\nsamples = 28\nvalue = [0, 0, 0, 28, 0]\nclass = drugX'),
 T